#Mount Google Drive

#Sales Data Analysis

1. Data Loading and Merging: Since the data is split across multiple files, we need to combine them into a single DataFrame.

2. Data Cleaning: We'll look for any missing values or anomalies in the dataset and handle them.

3. Data Exploration:
- Overview of the data.
- Summary statistics.
- Monthly sales analysis.
- Best selling products.
- Time of the day with the highest sales.

4. Data Visualization: Visualize our findings using various plots.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

#Step 1: Data Loading and Merging

In [ ]:
# Specify the path to the directory containing the CSV files
path = '/kaggle/input/year-2019-sales-data'

# Listing all files in the specified directory
files = os.listdir(path)

# Read each CSV file and concatenate them into a single DataFrame
df = pd.concat([pd.read_csv(os.path.join(path, file)) for file in files], ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull()

In [ ]:
df[df.duplicated()==True].sort_values(by='Order ID')

#Step 2: Data Cleaning

In [ ]:
dfnew = df.drop_duplicates()

In [ ]:
dfnew.sort_values(by='Order ID').head(5)

In [ ]:
dfnew.dtypes

In [ ]:
# Drop rows with NaN values
dfnew = dfnew.dropna(how="any")

# Check the number of rows after dropping NaN values
rows_after_cleaning = dfnew.shape[0]

rows_after_cleaning


In [ ]:
# Convert "Quantity Ordered" and "Price Each" to numeric data types
# Rows causing errors during conversion will be removed
dfnew = dfnew[dfnew["Quantity Ordered"].apply(lambda x: x.isnumeric())]
dfnew = dfnew[dfnew["Price Each"].apply(lambda x: x.replace('.', '', 1).isdigit())]

dfnew["Quantity Ordered"] = pd.to_numeric(dfnew["Quantity Ordered"])
dfnew["Price Each"] = pd.to_numeric(dfnew["Price Each"])

# Confirm the data types of the columns
dfnew.dtypes

In [ ]:
# Extracting month from "Order Date" column
dfnew['Month'] = dfnew['Order Date'].str[:2].astype(int)

# Extracting city from "Purchase Address" column
dfnew['City'] = dfnew['Purchase Address'].apply(lambda x: x.split(',')[1].strip())

# Display the first few rows with the new columns
dfnew.head()

- There are 19 unique products in the dataset. The products include various electronic items and accessories such as headphones, phones, batteries, monitors, laptops, and more

- Sales data covers 9 unique cities, which include major cities like Los Angeles, Dallas, Seattle, San Francisco, and Boston, among others.

#Step 3: Data Exploration

- Overview of the data: Get an understanding of the different products, cities, etc.
- Summary statistics: Basic statistics for numeric columns.
- Monthly sales analysis: Total sales for each month.
- Best selling products: Products with the highest sales.
- Time of the day with the highest sales: Analyzing when most sales occur.
- City wide sales: Sales by city
- Product with the Highest Orders Each Month
- Products Sold Together

In [ ]:
# Overview of unique products and cities in the dataset
unique_products = dfnew["Product"].unique()
unique_cities = dfnew["City"].unique()

unique_products, unique_cities

In [ ]:
# Summary statistics for numeric columns
summary_statistics = dfnew.describe()

summary_statistics

Quantity Ordered:
Mean: ~1.12

Standard Deviation: 0.44
Min: 1
Max: 9

Price Each:
Mean: ~$184.51

Standard Deviation: $332.84
Min: $2.99
Max: $1700.00

Month:
Mean: ~7.05 (This indicates the data is relatively uniformly spread across the year.)

Min: 1 (January)
Max: 12 (December)

In [ ]:
# Calculate total sales for each transaction
dfnew['Total Sales'] = dfnew['Quantity Ordered'] * dfnew['Price Each']

# Monthly sales analysis
monthly_sales = dfnew.groupby('Month').sum()['Total Sales']

monthly_sales

Monthly Sales Analysis:
The total sales for each month in 2019 are:

- January: $1,821,413.16
- February: $2,200,078.08
- March: $2,804,973.35
- April: $3,389,217.98
- May: $3,150,616.23
- June: $2,576,280.15
- July: $2,646,461.32
- August: $2,241,083.37
- September: $2,094,465.69
- October: $3,734,777.86
- November: $3,197,875.05
- December: $4,608,295.70

In [ ]:
monthly_sales.plot()

In [ ]:
# Products with the highest sales
product_sales = dfnew.groupby('Product').sum()['Total Sales']

# Sort products by total sales
sorted_product_sales = product_sales.sort_values(ascending=False)

sorted_product_sales

In [ ]:
# Convert "Order Date" to datetime type
dfnew['Order Date'] = pd.to_datetime(dfnew['Order Date'], errors='coerce')

# Extract hour from the order date
dfnew['Hour'] = dfnew['Order Date'].dt.hour

# Sum sales for each hour
hourly_sales = dfnew.groupby('Hour')['Total Sales'].sum()

hourly_sales

###Product with the Highest Orders Each Month

In [ ]:
product_monthly = dfnew.groupby(['Month', 'Product']).size().reset_index(name='Order Count')
top_products_each_month = product_monthly.loc[product_monthly.groupby('Month')['Order Count'].idxmax()]
print(top_products_each_month)

###Products Sold Together

In [ ]:
dfnew['Order ID'] = dfnew['Order ID'].astype('str')
grouped_orders = dfnew[dfnew['Order ID'].duplicated(keep=False)].copy()
grouped_orders['Grouped'] = grouped_orders.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
grouped_orders = grouped_orders[['Order ID', 'Grouped']].drop_duplicates()

# Display common combinations
grouped_orders['Grouped'].value_counts().head(10)

#Step 4: Data Visualization

Visualizing the data can help in understanding these trends more intuitively. Let's create some plots to visualize.

1. Monthly sales.
2. Best selling products by total sales.
3. Sales distribution throughout the day.

In [ ]:
import matplotlib.pyplot as plt

# Plot for Monthly Sales
months = range(1, 13)
plt.figure(figsize=(12, 6))
plt.bar(monthly_sales.index, monthly_sales.values, color='skyblue')
plt.xticks(months)
plt.title('Monthly Sales in 2019')
plt.xlabel('Month')
plt.ylabel('Sales in USD ($)')
plt.show()

- December has the highest sales, followed closely by October and April.

- There's a noticeable dip in sales in the months of June through September, with a significant rise in October.

In [ ]:
# Plotting top selling products
plt.figure(figsize=(14, 10))
plt.barh(sorted_product_sales.index, sorted_product_sales.values, color='lightgreen')
plt.title('Top Selling Products in 2019')
plt.xlabel('Sales in USD ($) millions')
plt.ylabel('Product')
plt.gca().invert_yaxis()  # To display the product with the highest sales at the top
plt.grid(axis='x')
plt.show()

MacBook Pro has the highest sales generating revenue above 8 millions USD, followed by iPhone and ThinkPad Laptops with sales above 5 million USD.

In [ ]:
# Plotting hourly sales
plt.figure(figsize=(8, 4))
sns.lineplot(x=hourly_sales.index, y=hourly_sales.values)
plt.title('Hourly Sales in 2019')
plt.xlabel('Hour')
plt.ylabel('Revenue in USD ($)')
plt.grid(True)
plt.show()

Sales increase from 7am to 8pm.

In [ ]:
# Quantity of Each Product Sold
product_quantity = dfnew.groupby('Product')['Quantity Ordered'].sum()
product_quantity.plot(kind='bar', figsize=(14, 7), title='Quantity of Each Product Sold')

AAA Battery, AA Battery, USB-C Charging Cable,Lightning Charging Cable, Wired Headphones, Apple Airpods Headphones and Bose SoundSport Headphones are the most commonly sold products.

In [ ]:
# Top 10 product combinations
product_combinations = grouped_orders['Grouped'].value_counts().head(10)

plt.figure(figsize=(14, 8))
product_combinations.plot(kind='barh', color='lightblue')
plt.title('Top 10 Most Common Product Combinations')
plt.xlabel('Number of Times Ordered Together')
plt.ylabel('Product Combinations')
plt.gca().invert_yaxis()  # Display the most common combination at the top
plt.grid(axis='x')
plt.show()

iPhone and Lightning Charging Cable are the most common product combinations, followed by Google Phone and USB-C Charging Cable.

###City wide sales

In [ ]:
city_sales = dfnew.groupby('City')['Total Sales'].sum()
city_sales.plot(kind='bar', figsize=(12, 6), title='Sales by City')

San Francisco has the highest sales, followed by Los Angeles and New York City.

In [ ]:
dfnew['DayOfWeek'] = pd.to_datetime(dfnew['Order Date']).dt.dayofweek
weekday_sales = dfnew.groupby('DayOfWeek')['Total Sales'].sum()
weekday_sales.plot(kind='line', figsize=(12, 6), title='Sales Trend by Day of Week')

Monday is 0, while Sunday is 6.

Tuesday has the highest sales & Thursday has the lowest.